In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))  

In [2]:
import cirq
from qudit_cirq.circuit_builder import create_circuit
from qudit_cirq.utils import format_out, printVector, tensor_product
from qudit_cirq.qudit import quditXGate, quditZGate, quditHGate, quditCNOTGate, state_vector

In [3]:
d =4 # dimensions

In [4]:
circuit, qudits, qudit_order = create_circuit(
    (2, quditHGate, "qudit1"),                      
    (3, quditXGate, "qudit2"),                      
    (2, quditCNOTGate, ["qudit1", "qudit3"]),       
    (3, quditCNOTGate, ["qudit2", "qudit4"])        
)

print("Circuit:")
print(circuit.to_text_diagram(qubit_order=qudit_order))

Circuit:
                          ┌──────────────────┐
qudit1 (d=2): ───H(d=2)────┬─────────────────────
                           │
qudit2 (d=3): ───X(d=3)────┼────────┬────────────
                           │        │
qudit3 (d=2): ─────────────CNOT(d=2)┼────────────
                                    │
qudit4 (d=3): ──────────────────────CNOT(d=3)────
                          └──────────────────┘


In [5]:
d = 3

In [6]:
state_qudit1 = state_vector(d, 0)  # Initializes |0⟩ for qudit1
state_qudit2 = state_vector(d, 0)  # Initializes |1⟩ for qudit2

In [7]:
state_qudit1

array([1.+0.j, 0.+0.j, 0.+0.j])

In [8]:
state_qudit2

array([1.+0.j, 0.+0.j, 0.+0.j])

In [9]:
initial_state = tensor_product(state_qudit1, state_qudit2)

In [10]:
initial_state

array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j])

In [11]:
# Build the circuit
circuit, qudits, qudit_order = create_circuit(
    d,  
    (quditHGate, "qudit1"),                   
    (quditCNOTGate, ["qudit1", "qudit2"])      
)

print("Circuit:")
print(circuit)

Circuit:
qudit1 (d=3): ───H(d=3)───┬───────────
                          │
qudit2 (d=3): ────────────CNOT(d=3)───


In [12]:
simulator = cirq.Simulator()

In [13]:
result = simulator.simulate(circuit, initial_state=initial_state)
final_state = result.final_state_vector
dimensions = [qudit.dimension for qudit in qudit_order]
print("Final state vector:")
print(format_out(result.final_state_vector))

Final state vector:
[['0.6' '0.0' '0.0' '0.0' '0.6' '0.0' '0.0' '0.0' '0.6']]


In [14]:
printVector(final_state, dimensions, qudits=qudit_order)


Final state vector:
|00⟩: (0.5773502588272095+0j)
|11⟩: (0.5773502588272095+0j)
|22⟩: (0.5773502588272095+0j)


In [15]:
d = 3


control_qudit = cirq.NamedQid("control", dimension=d)
target_qudit = cirq.NamedQid("target", dimension=d)


cnot_gate = quditCNOTGate(d)
unitary_matrix = cnot_gate._unitary_()

circuit = cirq.Circuit()
circuit.append(cnot_gate(control_qudit, target_qudit))

print(circuit)

control (d=3): ───┬───────────
                  │
target (d=3): ────CNOT(d=3)───


In [16]:
#print("Unitary matrix (X) (d={}):\n".format(d), unitary_matrix)
formatted_matrix = format_out(unitary_matrix)
print(" Unitary matrix for the CNOT gate (d={}):\n".format(d), formatted_matrix.T)

 Unitary matrix for the CNOT gate (d=3):
 [['1.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0']
 ['0.0' '1.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0']
 ['0.0' '0.0' '1.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0']
 ['0.0' '0.0' '0.0' '0.0' '1.0' '0.0' '0.0' '0.0' '0.0']
 ['0.0' '0.0' '0.0' '0.0' '0.0' '1.0' '0.0' '0.0' '0.0']
 ['0.0' '0.0' '0.0' '1.0' '0.0' '0.0' '0.0' '0.0' '0.0']
 ['0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '1.0']
 ['0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '1.0' '0.0' '0.0']
 ['0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '1.0' '0.0']]
